# Execute EfficientDet TTA with ODAch
This is a example to conduct TTA, using a tta-tool **odach** with EfficientDet.

Odach page is bellow, please check it out for details!

https://github.com/kentaroy47/ODA-Object-Detection-ttA

In [ ]:
!pip install --no-deps '../input/timm-package/timm-0.1.26-py3-none-any.whl' > /dev/null
!pip install --no-deps '../input/pycocotools/pycocotools-2.0-cp37-cp37m-linux_x86_64.whl' > /dev/null

In [ ]:
import sys
sys.path.insert(0, "../input/timm-efficientdet-pytorch")
sys.path.insert(0, "../input/omegaconf")
sys.path.insert(0, "../input/weightedboxesfusion")
# Import odach
sys.path.insert(0, "../input/odachkaggle/ODA-Object-Detection-ttA-main/")

import odach as oda

from ensemble_boxes import *
import torch
import numpy as np
import pandas as pd
from glob import glob
from torch.utils.data import Dataset,DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import cv2
import gc
from matplotlib import pyplot as plt
from effdet import get_efficientdet_config, EfficientDet, DetBenchEval
from effdet.efficientdet import HeadNet

In [ ]:
def get_valid_transforms():
    return A.Compose([
            A.Resize(height=512, width=512, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)

In [ ]:
DATA_ROOT_PATH = '../input/global-wheat-detection/test'

class DatasetRetriever(Dataset):

    def __init__(self, image_ids, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = cv2.imread(f'{DATA_ROOT_PATH}/{image_id}.jpg', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0
        if self.transforms:
            sample = {'image': image}
            sample = self.transforms(**sample)
            image = sample['image']
        return image, image_id

    def __len__(self) -> int:
        return self.image_ids.shape[0]

In [ ]:
dataset = DatasetRetriever(
    image_ids=np.array([path.split('/')[-1][:-4] for path in glob(f'{DATA_ROOT_PATH}/*.jpg')]),
    transforms=get_valid_transforms()
)

def collate_fn(batch):
    return tuple(zip(*batch))

data_loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    drop_last=False,
    #collate_fn=collate_fn
)

# Load effdet

In [ ]:
def load_net(checkpoint_path):
    config = get_efficientdet_config('tf_efficientdet_d5')
    net = EfficientDet(config, pretrained_backbone=False)

    config.num_classes = 1
    config.image_size=512
    net.class_net = HeadNet(config, num_outputs=config.num_classes, norm_kwargs=dict(eps=.001, momentum=.01))

    checkpoint = torch.load(checkpoint_path)
    net.load_state_dict(checkpoint['model_state_dict'])

    del checkpoint
    gc.collect()

    net = DetBenchEval(net, config)
    net.eval();
    return net.cuda()

net = load_net('../input/wheat-effdet5-fold0-best-checkpoint/fold0-best-all-states.bin')

## Wrap effdet outputs for odach

In [ ]:
class wrap_effdet:
    def __init__(self, model):
        self.model = model
    
    def __call__(self, img, score_threshold=0.22):
        det = self.model(img, torch.tensor([1]*images.shape[0]).float().cuda())
        predictions = []
        for i in range(img.shape[0]):
            boxes = det[i][:,:4]  
            scores = det[i][:,4]
            npscore = scores.detach().cpu().numpy()
            indexes = np.where(npscore > score_threshold)[0]
            boxes = boxes[indexes]
            boxes[:, 2] = boxes[:, 2] + boxes[:, 0]
            boxes[:, 3] = boxes[:, 3] + boxes[:, 1]
            # clamp boxes
            boxes = boxes.clamp(0, 511)
            predictions.append({
                'boxes': boxes[indexes], #/(img.shape[2]-1),
                'scores': scores[indexes],
                "labels": torch.from_numpy(np.ones_like(npscore[indexes])).cuda()
            })
        return predictions

In [ ]:
# wrap effdet
effdet = wrap_effdet(net.eval())
# Declare TTA variations
tta = [oda.HorizontalFlip(), oda.VerticalFlip(), oda.Rotate90()]
# Declare scales to tta
scale = [1]
# wrap model and tta
tta_model = oda.TTAWrapper(effdet, tta, scale)

# Inference on a image

In [ ]:
import matplotlib.pyplot as plt

for j, (images, image_ids) in enumerate(data_loader):
    break

images2 = images.cuda().float()
boxes, scores, labels = tta_model(images2)

# Visualize
i = 0
sample = images[i].permute(1,2,0).cpu().numpy()
sample = cv2.UMat(sample).get()
boxes = (boxes*(images.shape[2]-1)).astype(np.int32).clip(min=0, max=511)

fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (1,0,0), 2)
    
ax.set_axis_off()
ax.imshow(sample)

# Submit!

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))
    return " ".join(pred_strings)

In [ ]:
results = []

for images, image_ids in data_loader:
    images = images.cuda()
    i=0
    boxes, scores, labels = tta_model(images)
    boxes = (boxes*1023).astype(np.int32).clip(min=0, max=1023)
    image_id = image_ids[i]
    
    if len(data_loader)<30:
        i = 0
        fig, ax = plt.subplots(1, 1, figsize=(16, 8))
        sample = images[i].permute(1,2,0).cpu().numpy()
        sample = cv2.UMat(sample).get()
        for box in boxes:
            box = (box/2).astype(np.int32)
            cv2.rectangle(sample, (box[0], box[1]), (box[2], box[3]), (1, 0, 0), 2)

        ax.set_axis_off()
        ax.imshow(sample);

    boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    

    result = {
        'image_id': image_id,
        'PredictionString': format_prediction_string(boxes, scores)
    }
    results.append(result)

In [ ]:
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
test_df.to_csv('submission.csv', index=False)
test_df.head()